<a href="https://colab.research.google.com/github/manikanta-eng/HPC/blob/main/hpc_lab_5_2303A51271.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**section 1  Serial Recursive Fibonacci**

In [7]:
import time
def fib(n):
    if n <= 1:
        return n
    return fib(n - 1) + fib(n - 2)
if __name__ == "__main__":
    n = int(input("Enter the value of n: "))

    start_time = time.time()
    result = fib(n)
    end_time = time.time()

    execution_time = end_time - start_time

    print(f"Fibonacci({n}) = {result}")
    print(f"Execution Time: {execution_time:.6f} seconds")


Enter the value of n: 10
Fibonacci(10) = 55
Execution Time: 0.000015 seconds


**Section 2: OpenMP Task-based Fibonacci (Basic
Tasking)**

In [9]:
import time
from numba import njit
from concurrent.futures import ThreadPoolExecutor

@njit
def fib_serial(n):
  if n <= 1:
    return n
  return fib_serial(n - 1) + fib_serial(n - 2)

def fib_parallel(n, threshold=20):
  if n <= 1:
    return n
  if n <= threshold:
    return fib_serial(n)

  with ThreadPoolExecutor() as executor:
    future1 = executor.submit(fib_parallel, n - 1, threshold)
    future2 = executor.submit(fib_parallel, n - 2, threshold)
    return future1.result() + future2.result()

def main():
  n = int(input("Enter the value of n: "))

  # Serial execution
  start_time_serial = time.time()
  serial_result = fib_serial(n)
  serial_time = time.time() - start_time_serial
  print(f"\nSerial Fibonacci: {serial_result}")
  print(f"Serial Execution Time: {serial_time:.6f} seconds")

  # Parallel execution
  start_time_parallel = time.time()
  parallel_result = fib_parallel(n)
  parallel_time = time.time() - start_time_parallel
  print(f"\nParallel Fibonacci: {parallel_result}")
  print(f"Parallel Execution Time: {parallel_time:.6f} seconds")

if __name__ == "__main__":
    main()

Enter the value of n: 10

Serial Fibonacci: 55
Serial Execution Time: 1.643273 seconds

Parallel Fibonacci: 55
Parallel Execution Time: 0.000005 seconds


**Section 3: Task Creation and Synchronization using
taskwait**

In [11]:
import time
from numba import njit
from concurrent.futures import ThreadPoolExecutor

@njit
def fib_serial(n):
  if n <= 1:
    return n
  return fib_serial(n - 1) + fib_serial(n - 2)

def fib_taskwait(n, executor, threshold=20):

  if n <= 1:
    return n
  if n <= threshold:
    return fib_serial(n)

  future1 = executor.submit(fib_taskwait, n - 1, executor, threshold)
  future2 = executor.submit(fib_taskwait, n - 2, executor, threshold)

  x = future1.result()
  y = future2.result()

  return x + y

def main():
  n = int(input("Enter the value of n: "))
  start = time.time()

  with ThreadPoolExecutor() as executor:
    result = fib_taskwait(n, executor)

  end = time.time()
  print(f"\nParallel Fibonacci: {result}")
  print(f"Execution Time: {end - start:.6f} seconds")


if __name__ == "__main__":
  main()

Enter the value of n: 10

Parallel Fibonacci: 55
Execution Time: 0.085004 seconds


**Section 4: Performance Analysis of OpenMP Tasking**

In [12]:
import time
from concurrent.futures import ProcessPoolExecutor

THRESHOLD = 20

def fib_serial(n):
    if n <= 1:
        return n
    return fib_serial(n - 1) + fib_serial(n - 2)

def fib_task(n):
    if n <= 1:
        return n


    if n < THRESHOLD:
        return fib_serial(n)

    with ProcessPoolExecutor() as executor:
        f1 = executor.submit(fib_task, n - 1)
        f2 = executor.submit(fib_task, n - 2)
        return f1.result() + f2.result()

if __name__ == "__main__":
    test_values = [20, 25, 30, 35]

    print("Task-Based Fibonacci Performance Analysis\n")

    for n in test_values:
        start = time.perf_counter()
        result = fib_task(n)
        end = time.perf_counter()

        print(f"Fibonacci({n}) = {result}")
        print(f"Execution Time: {end - start:.6f} seconds\n")


Task-Based Fibonacci Performance Analysis

Fibonacci(20) = 6765
Execution Time: 0.035449 seconds

Fibonacci(25) = 75025
Execution Time: 0.531472 seconds

Fibonacci(30) = 832040
Execution Time: 10.649652 seconds

Fibonacci(35) = 9227465
Execution Time: 161.135480 seconds



**Section 5: Recursive Divide-and-Conquer Sum using
OpenMP Tasks**

In [13]:
import time
from concurrent.futures import ProcessPoolExecutor

def chunk_sum(chunk):
    return sum(chunk)

if __name__ == "__main__":
    arr = list(range(1, 1_000_001))
    num_workers = 4
    chunk_size = len(arr) // num_workers

    chunks = [
        arr[i:i + chunk_size]
        for i in range(0, len(arr), chunk_size)
    ]

    start = time.perf_counter()

    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        partial_sums = executor.map(chunk_sum, chunks)

    total_sum = sum(partial_sums)
    end = time.perf_counter()

    print("Loop-Based Parallel Sum")
    print("Sum =", total_sum)
    print("Execution Time:", end - start)


Loop-Based Parallel Sum
Sum = 500000500000
Execution Time: 0.19117357499999343
